In [1]:
import pandas as pd
import numpy as np

train_cols = ['ip', 'app', 'device', 'os', 'channel', 'click_time', 'is_attributed']

dtypes = {
        'ip'            : 'uint32',
        'app'           : 'uint16',
        'device'        : 'uint16',
        'os'            : 'uint16',
        'channel'       : 'uint16',
        'is_attributed' : 'uint8',
        'click_id'      : 'uint32'
        }
        

data = pd.read_csv('../input/train.csv', header=0,usecols=train_cols,parse_dates=["click_time"], dtype=dtypes)

print('len read:',len(data))

len read: 184903890


In [2]:
#attributed_data = data.query('is_attributed == 1')

#print('len of attributed:',len(attributed_data))

len of attributed: 456846


In [3]:
print(1.0 - 456846/184903890)

0.997529278589001


In [4]:
print('sampling data')
#data = data.set_index('ip').loc[lambda x: (x.index + 401) % 10 == 0].reset_index()

data['hour'] = data["click_time"].dt.hour.astype('uint8')
data['day'] = data["click_time"].dt.day.astype('uint8')

data.drop('click_time', inplace=True, axis=1)

sampling data


In [5]:
#data.drop('click_time', inplace=True, axis=1)

print('added hour and day')

cvr_columns_lists = [['ip','device'],['app','channel']]

for cvr_columns in cvr_columns_lists:
    sta_ft = data[cvr_columns + ['hour','day','is_attributed']].\
        groupby(cvr_columns + ['day','hour'])[['is_attributed']].mean().reset_index()
    print(sta_ft.describe())
    sta_ft.info()

    sta_ft['day'] = sta_ft['day'] +1
    
    new_col_name = '_'.join(cvr_columns + ['cvr'])
    sta_ft = sta_ft.rename(columns={'is_attributed':new_col_name})
    data= data.merge(sta_ft, on=cvr_columns + ['day','hour'], how='left')

    data[new_col_name] = data[new_col_name].astype('float32')

    import gc
    del sta_ft
    gc.collect()

    print(data)
    print(data.describe())
    data.info()

data.to_csv('train_with_cvr.csv.gzip', index=False, compression='compression')

added hour and day
                 ip        device           day          hour  is_attributed
count  4.597732e+06  4.597732e+06  4.597732e+06  4.597732e+06   4.597732e+06
mean   1.315670e+05  2.342364e+02  7.744687e+00  1.035701e+01   4.037218e-02
std    9.299802e+04  8.434220e+02  9.240044e-01  6.502230e+00   1.805420e-01
min    1.000000e+00  0.000000e+00  6.000000e+00  0.000000e+00   0.000000e+00
25%    5.593000e+04  1.000000e+00  7.000000e+00  5.000000e+00   0.000000e+00
50%    1.122340e+05  1.000000e+00  8.000000e+00  1.000000e+01   0.000000e+00
75%    1.928090e+05  1.000000e+00  9.000000e+00  1.500000e+01   0.000000e+00
max    3.647780e+05  4.227000e+03  9.000000e+00  2.300000e+01   1.000000e+00
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4597732 entries, 0 to 4597731
Data columns (total 5 columns):
ip               uint64
device           uint64
day              uint64
hour             uint64
is_attributed    float64
dtypes: float64(1), uint64(4)
memory usage: 175.4 MB
  

                app       channel           day          hour  is_attributed
count  44911.000000  44911.000000  44911.000000  44911.000000   44911.000000
mean      94.790875    265.504620      7.717731     10.931041       0.074681
std      124.891862    130.255017      0.932940      6.727209       0.200645
min        0.000000      0.000000      6.000000      0.000000       0.000000
25%       12.000000    145.000000      7.000000      5.000000       0.000000
50%       29.000000    274.000000      8.000000     11.000000       0.000000
75%      148.000000    347.000000      8.000000     16.000000       0.001783
max      768.000000    500.000000      9.000000     23.000000       1.000000
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44911 entries, 0 to 44910
Data columns (total 5 columns):
app              44911 non-null uint64
channel          44911 non-null uint64
day              44911 non-null uint64
hour             44911 non-null uint64
is_attributed    44911 non-null float64
dty

                 ip           app        device            os       channel  \
count  1.849039e+08  1.849039e+08  1.849039e+08  1.849039e+08  1.849039e+08   
mean   9.087604e+04  1.201131e+01  2.172325e+01  2.267702e+01  2.685789e+02   
std    6.952789e+04  1.480521e+01  2.593326e+02  5.525282e+01  1.295882e+02   
min    1.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   
25%    4.024500e+04  3.000000e+00  1.000000e+00  1.300000e+01  1.400000e+02   
50%    7.962200e+04  1.200000e+01  1.000000e+00  1.800000e+01  2.580000e+02   
75%    1.182470e+05  1.500000e+01  1.000000e+00  1.900000e+01  3.790000e+02   
max    3.647780e+05  7.680000e+02  4.227000e+03  9.560000e+02  5.000000e+02   

       is_attributed          hour           day  ip_device_cvr  \
count   1.849039e+08  1.849039e+08  1.849039e+08   1.011945e+08   
mean    2.470721e-03  9.298776e+00  7.863532e+00   1.231440e-03   
std     4.964491e-02  6.171641e+00  8.899349e-01   1.330432e-02   
min     0.000000e+00

In [6]:
print(data.describe())

                 ip           app        device            os       channel  \
count  1.849039e+08  1.849039e+08  1.849039e+08  1.849039e+08  1.849039e+08   
mean   9.087604e+04  1.201131e+01  2.172325e+01  2.267702e+01  2.685789e+02   
std    6.952789e+04  1.480521e+01  2.593326e+02  5.525282e+01  1.295882e+02   
min    1.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   
25%    4.024500e+04  3.000000e+00  1.000000e+00  1.300000e+01  1.400000e+02   
50%    7.962200e+04  1.200000e+01  1.000000e+00  1.800000e+01  2.580000e+02   
75%    1.182470e+05  1.500000e+01  1.000000e+00  1.900000e+01  3.790000e+02   
max    3.647780e+05  7.680000e+02  4.227000e+03  9.560000e+02  5.000000e+02   

       is_attributed          hour           day  ip_device_cvr  \
count   1.849039e+08  1.849039e+08  1.849039e+08   1.011945e+08   
mean    2.470721e-03  9.298776e+00  7.863532e+00   1.231440e-03   
std     4.964491e-02  6.171641e+00  8.899349e-01   1.330432e-02   
min     0.000000e+00